# Day 78 of 100 days of code

**Objective:** Do higher film budgets lead to more box office revenue? Should a movie studio spend more on a film to make more? Let's find out if there's a relationship using the movie budgets and financial performance data scraped from [the-numbers.com](https://www.the-numbers.com/movie/budgets) on **May 1st, 2018**. 

<img src=https://i.imgur.com/kq7hrEh.png height=150>


**Today you'll learn:**

- How to use a popular data visualisation library called Seaborn
- How to run and interpret a linear regression with scikit-learn
- How to plot a regression a scatter plot to visualise relationships in the data
- How to add a third dimension to a scatter plot to create a bubble chart
- How to cleverly use floor division // to convert your data

# Import Statements

In [191]:
import pandas as pd
import matplotlib.pyplot as plt

# Notebook Presentation

In [192]:
pd.options.display.float_format = '{:,.2f}'.format
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# Read the Data

In [193]:
data = pd.read_csv('cost_revenue_dirty.csv')
data[:5]

,Rank,Release_Date,Movie_Title,USD_Production_Budget,USD_Worldwide_Gross,USD_Domestic_Gross
0,5293,8/2/1915,The Birth of a Nation,"$110,000","$11,000,000","$10,000,000"
1,5140,5/9/1916,Intolerance,"$385,907",$0,$0
2,5230,12/24/1916,"20,000 Leagues Under the Sea","$200,000","$8,000,000","$8,000,000"
3,5299,9/17/1920,Over the Hill to the Poorhouse,"$100,000","$3,000,000","$3,000,000"
4,5222,1/1/1925,The Big Parade,"$245,000","$22,000,000","$11,000,000"


# Explore and Clean the Data

**Challenge**: Answer these questions about the dataset:

**1. How many rows and columns does the dataset contain?** (5391, 6)

**2. Are there any NaN values present?** No

**3. Are there any duplicate rows?** No

**4. What are the data types of the columns?** See below

In [194]:
print(data.shape)
print("Any NaN values?", data.isna().values.any())  # No NaN values
print("Any dupilcated rows?", data.duplicated().values.any())  # No duplicated rows
data.dtypes

(5391, 6)
Any NaN values? False
Any dupilcated rows? False


Rank                      int64
Release_Date             object
Movie_Title              object
USD_Production_Budget    object
USD_Worldwide_Gross      object
USD_Domestic_Gross       object
dtype: object

In [195]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5391 entries, 0 to 5390
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Rank                   5391 non-null   int64 
 1   Release_Date           5391 non-null   object
 2   Movie_Title            5391 non-null   object
 3   USD_Production_Budget  5391 non-null   object
 4   USD_Worldwide_Gross    5391 non-null   object
 5   USD_Domestic_Gross     5391 non-null   object
dtypes: int64(1), object(5)
memory usage: 252.8+ KB


### Data Type Conversions

**Challenge**: Convert the `USD_Production_Budget`, `USD_Worldwide_Gross`, and `USD_Domestic_Gross` columns to a numeric format by removing `$` signs and `,`. 
<br>
<br>
Note that *domestic* in this context refers to the United States.

In [196]:
columns_to_clean = ["USD_Production_Budget", "USD_Worldwide_Gross", "USD_Domestic_Gross"]
# Remove "$" and ","
data[columns_to_clean] = data[columns_to_clean].replace(r"\$|\,", "", regex=True)
data[:5]

,Rank,Release_Date,Movie_Title,USD_Production_Budget,USD_Worldwide_Gross,USD_Domestic_Gross
0,5293,8/2/1915,The Birth of a Nation,110000,11000000,10000000
1,5140,5/9/1916,Intolerance,385907,0,0
2,5230,12/24/1916,"20,000 Leagues Under the Sea",200000,8000000,8000000
3,5299,9/17/1920,Over the Hill to the Poorhouse,100000,3000000,3000000
4,5222,1/1/1925,The Big Parade,245000,22000000,11000000


In [197]:
# Convert the budget and revenue columns to numeric type
for col in columns_to_clean:
    data[col] = pd.to_numeric(data[col])
data.dtypes

Rank                      int64
Release_Date             object
Movie_Title              object
USD_Production_Budget     int64
USD_Worldwide_Gross       int64
USD_Domestic_Gross        int64
dtype: object

**Challenge**: Convert the `Release_Date` column to a Pandas Datetime type. 

In [198]:
data["Release_Date"] = pd.to_datetime(data["Release_Date"], format="%m/%d/%Y")
print(data.dtypes)
data[:2]

Rank                              int64
Release_Date             datetime64[ns]
Movie_Title                      object
USD_Production_Budget             int64
USD_Worldwide_Gross               int64
USD_Domestic_Gross                int64
dtype: object


,Rank,Release_Date,Movie_Title,USD_Production_Budget,USD_Worldwide_Gross,USD_Domestic_Gross
0,5293,1915-08-02,The Birth of a Nation,110000,11000000,10000000
1,5140,1916-05-09,Intolerance,385907,0,0


### Descriptive Statistics

We see from the description table that there are 5391 movies in the dataset. They are ranked from 1 to 5391. 

**Challenge**: 

**1. What is the average production budget of the films in the data set?**  

- 31,113,738&nbsp;USD

**2. What is the average worldwide gross revenue of films?** 

- 88,855,421.96&nbsp;USD (_no: gjennomsnittlig bruttoinntekt/omsetning på verdensbasis_)

**3. What were the minimums for worldwide and domestic revenue?** 

- 0&nbsp;USD for both worldwide and domestic revenue

**4. Are the bottom 25% of films actually profitable or do they lose money?** 

- The soultion for this challenge (provided in the course) treats "the bottom quartile" as a group of films:
_"... But quite a lot of films lose money too. In fact, all the films in the bottom quartile lose money, since the average cost is $5 million and they only bring in $3.8m in worldwide revenue"_. This is not a correct interpretation of the description table below (made with `.describe()`). This statement is assuming that all the values in the row labeled "25%" (25th percentile) are related to each other, but the percentile values are not related because they are calculated for each column in the DataFrame. This becomes clear if we look at the row labeled "min". The values in this row shows us the lowest value occuring in each column in the DataFrame. The minimum value for "Rank" is 1, which is the rank of the movie Avatar. The lowest production budget is 1100 USD, which is the budget of the movie "My Date With Drew".

- To answer the question I assume that "the bottom 25% of films" refers to the 25% lowest (worst) ranked films, i.e., movies with rank 4044 or higher. Se further explanation and code below.

**5. What are the highest production budget and highest worldwide gross revenue of any film?** 

- The highest production budget is 425,000,000&nbsp;USD, and the highest worldwide gross revenue is 2,783,918,982&nbsp;USD.

**6. How much revenue did the lowest and highest budget films make?** 

- The movie with the lowest production budget (1,100 USD) is "My Date With Drew" which made 181,041 USD.
- The movie with the highest production budget (425,000,000 USD) is "Avatar" which made 2,783,918,982 USD.

In [199]:
data.describe()

,Rank,Release_Date,USD_Production_Budget,USD_Worldwide_Gross,USD_Domestic_Gross
count,"5,391.00",5391,"5,391.00","5,391.00","5,391.00"
mean,"2,696.00",2003-09-19 15:02:02.203672704,"31,113,737.58","88,855,421.96","41,235,519.44"
min,1.00,1915-08-02 00:00:00,"1,100.00",0.00,0.00
25%,"1,348.50",1999-12-02 12:00:00,"5,000,000.00","3,865,206.00","1,330,901.50"
50%,"2,696.00",2006-06-23 00:00:00,"17,000,000.00","27,450,453.00","17,192,205.00"
75%,"4,043.50",2011-11-23 00:00:00,"40,000,000.00","96,454,455.00","52,343,687.00"
max,"5,391.00",2020-12-31 00:00:00,"425,000,000.00","2,783,918,982.00","936,662,225.00"
std,"1,556.39",NaN,"40,523,796.88","168,457,757.00","66,029,346.27"


In [200]:
data[data["Rank"] == 1]

,Rank,Release_Date,Movie_Title,USD_Production_Budget,USD_Worldwide_Gross,USD_Domestic_Gross
3529,1,2009-12-18,Avatar,425000000,2783918982,760507625


By adding the column "USD_Profit" (difference between production budget and the worldwide gross revenue) we see that the **average profit is USD&nbsp;57,741,684.38**.

In [201]:
data["USD_Profit"] = data["USD_Worldwide_Gross"] - data["USD_Production_Budget"]
print(data[:3].to_string())
data.describe()

   Rank Release_Date                   Movie_Title  USD_Production_Budget  USD_Worldwide_Gross  USD_Domestic_Gross  USD_Profit
0  5293   1915-08-02         The Birth of a Nation                 110000             11000000            10000000    10890000
1  5140   1916-05-09                   Intolerance                 385907                    0                   0     -385907
2  5230   1916-12-24  20,000 Leagues Under the Sea                 200000              8000000             8000000     7800000


,Rank,Release_Date,USD_Production_Budget,USD_Worldwide_Gross,USD_Domestic_Gross,USD_Profit
count,"5,391.00",5391,"5,391.00","5,391.00","5,391.00","5,391.00"
mean,"2,696.00",2003-09-19 15:02:02.203672704,"31,113,737.58","88,855,421.96","41,235,519.44","57,741,684.38"
min,1.00,1915-08-02 00:00:00,"1,100.00",0.00,0.00,"-175,000,000.00"
25%,"1,348.50",1999-12-02 12:00:00,"5,000,000.00","3,865,206.00","1,330,901.50","-2,244,775.00"
50%,"2,696.00",2006-06-23 00:00:00,"17,000,000.00","27,450,453.00","17,192,205.00","8,122,000.00"
75%,"4,043.50",2011-11-23 00:00:00,"40,000,000.00","96,454,455.00","52,343,687.00","59,518,660.00"
max,"5,391.00",2020-12-31 00:00:00,"425,000,000.00","2,783,918,982.00","936,662,225.00","2,358,918,982.00"
std,"1,556.39",NaN,"40,523,796.88","168,457,757.00","66,029,346.27","140,972,329.93"


**Question 4. Are the bottom 25% of films actually profitable or do they lose money?**

I assume that "the bottom 25% of films" refers to the 25% lowest (worst) ranked films, i.e., movies with rank 4044 or higher. This number is taken from the 75th percentile row in the column "Rank", which is 4043.5. The number 4043.5 tells us that 75% of the movies have a "Rank" value equal to 4043.5 or lower (i.e. higher rated, because rank nr. 1 is best). Since the Rank is an integer, I round up to the closest integer 4044. Notice that we are interested in movies with rank-value higher or equal to the 75th percentile. If we wanted to look at the 25% highest rated movies, we would look at the movies ranging from rank 1 (the best) to 1349, the latter being the 25th percentile of the values in the column "Rank".

When looking at the description table for a DataFrame consisting of the 25% lowest (worst) ranked movies, we see that the average profit is about $7.4M, while the median profit is close to -$90K. This tells us that the lowest ranked movies could be profitable, but most of them (more than 50%) are not.

In [202]:
data["Rank"].duplicated().any()  
# No movies are at the same rank. Good to know.

False

In [216]:
# Create a DataFrame with the 25% lowest ranked films

# 75th percentile value in the "Rank" column
rank_q75 = data["Rank"].quantile(0.75)

# Filter data: include rows where "Rank" is equal or higher than the 75th percentile.
bottom_25p_movies = data.loc[data["Rank"] >= rank_q75]  
print(f"{bottom_25p_movies.shape = }")

bottom_25p_movies.describe()

bottom_25p_movies.shape = (1348, 7)


,Rank,Release_Date,USD_Production_Budget,USD_Worldwide_Gross,USD_Domestic_Gross,USD_Profit
count,"1,348.00",1348,"1,348.00","1,348.00","1,348.00","1,348.00"
mean,"4,717.50",2001-04-12 12:23:30.089020800,"1,874,027.95","9,299,180.18","5,982,015.03","7,425,152.23"
min,"4,044.00",1915-08-02 00:00:00,"1,100.00",0.00,0.00,"-5,000,000.00"
25%,"4,380.75",1999-03-13 18:00:00,"500,000.00","6,159.50",0.00,"-1,200,000.00"
50%,"4,717.50",2007-04-22 00:00:00,"1,500,000.00","636,005.50","257,153.50","-89,881.50"
75%,"5,054.25",2013-01-06 18:00:00,"3,000,000.00","7,146,872.50","4,051,902.25","4,713,997.50"
max,"5,391.00",2017-08-25 00:00:00,"5,000,000.00","390,525,192.00","198,680,470.00","386,625,192.00"
std,389.28,NaN,"1,471,007.51","27,187,087.57","16,655,558.08","27,034,074.86"


**Question 6. How much revenue did the lowest and highest budget films make?**

In [204]:
low_budj_movie = data.iloc[data['USD_Production_Budget'].idxmin()]
print(f"\nThe movie with the lowest production budget ({'{:,.0f}'.format(low_budj_movie['USD_Production_Budget'])} USD) is \"{low_budj_movie['Movie_Title']}\" which made {'{:,.0f}'.format(low_budj_movie['USD_Worldwide_Gross'])} USD.")


The movie with the lowest production budget (1,100 USD) is "My Date With Drew" which made 181,041 USD.


In [205]:
high_budj_movie = data.iloc[data['USD_Production_Budget'].idxmax()]
print(f"\nThe movie with the highest production budget ({'{:,.0f}'.format(high_budj_movie['USD_Production_Budget'])} USD) is \"{high_budj_movie['Movie_Title']}\" which made {'{:,.0f}'.format(high_budj_movie['USD_Worldwide_Gross'])} USD.")


The movie with the highest production budget (425,000,000 USD) is "Avatar" which made 2,783,918,982 USD.


# Investigating the Zero Revenue Films

**Challenge** How many films grossed $0 domestically (i.e., in the United States)? What were the highest budget films that grossed nothing?

- 512 films made $0 in the US.

On the first try of getting the top 5 list of zero-grossing films with the highest budget, we get the following: "Singularity", "Aquaman", "A Wrinkle in Time", "Amusement Park" and "Don Gato, el inicio de la pandilla". However, this is wrong. All the films on the list, except "Don Gato, el inicio de la pandilla" were released after this dataset was scraped (May 2018), and therefore their revenue could not have been known at the time.

A good example is Aquaman, which according to [wikipedia](https://en.wikipedia.org/wiki/Aquaman_(film)#:~:text=While%20the%20film%20received%20mixed,grossing%20film%20of%20all%20time.) was first released in November 2018, _"grossed $1.152 billion worldwide, making it the highest-grossing DCEU film, the highest-grossing film based on a DC Comics character, the fifth-highest-grossing film of 2018, and the 20th highest-grossing film of all time."_.

In [212]:
zero_rev_films = data[data["USD_Domestic_Gross"] == 0]
print(zero_rev_films.shape)
zero_rev_films[:2]

(512, 7)


,Rank,Release_Date,Movie_Title,USD_Production_Budget,USD_Worldwide_Gross,USD_Domestic_Gross,USD_Profit
1,5140,1916-05-09,Intolerance,385907,0,0,-385907
6,4630,1927-12-08,Wings,2000000,0,0,-2000000


In [214]:
zero_rev_films.sort_values("USD_Production_Budget", ascending=False)[:5]

,Rank,Release_Date,Movie_Title,USD_Production_Budget,USD_Worldwide_Gross,USD_Domestic_Gross,USD_Profit
5388,96,2020-12-31,Singularity,175000000,0,0,-175000000
5387,126,2018-12-18,Aquaman,160000000,0,0,-160000000
5384,321,2018-09-03,A Wrinkle in Time,103000000,0,0,-103000000
5385,366,2018-10-08,Amusement Park,100000000,0,0,-100000000
5090,556,2015-12-31,"Don Gato, el inicio de la pandilla",80000000,4547660,0,-75452340


To take the release date into account, we must at least exclude movies released after the data was scraped. I do not know much abouth how films are making money, but I guess that the total revenue is not made on the release date (? or maybe they sell the movie in advance?). Anyway, I think we would get a more correct picture if we exclude movies released in 2018.

In [228]:
zero_rev_films2 = data.loc[
    (data["USD_Domestic_Gross"] == 0) &
    (data["Release_Date"] < "2018-01-01")
]
print(zero_rev_films2.shape)
zero_rev_films2[:2]

(505, 7)


,Rank,Release_Date,Movie_Title,USD_Production_Budget,USD_Worldwide_Gross,USD_Domestic_Gross,USD_Profit
1,5140,1916-05-09,Intolerance,385907,0,0,-385907
6,4630,1927-12-08,Wings,2000000,0,0,-2000000


In [229]:
zero_rev_films2.sort_values("USD_Production_Budget", ascending=False)[:5]

,Rank,Release_Date,Movie_Title,USD_Production_Budget,USD_Worldwide_Gross,USD_Domestic_Gross,USD_Profit
5090,556,2015-12-31,"Don Gato, el inicio de la pandilla",80000000,4547660,0,-75452340
4294,566,2012-12-31,Astérix et Obélix: Au service de Sa Majesté,77600000,60680125,0,-16919875
5338,879,2017-04-08,The Dark Tower,60000000,0,0,-60000000
5058,880,2015-11-12,The Ridiculous 6,60000000,0,0,-60000000
4295,1230,2012-12-31,Foodfight!,45000000,73706,0,-44926294


"Don Gato, el inicio de la pandilla" is Spanish (I think), and "Astérix et Obélix: Au service de Sa Majesté" is French, and we can assume that they were released in French and Spanish speaking countries respectively, not making a single dolar in the US. That make sense.

However, at [the-numbers.com/movie/Dark-Tower](https://www.the-numbers.com/movie/Dark-Tower-The#tab=summary) (on January 5th, 2024) we see that "The Dark Tower (2017)" has a domestic gross of $50,701,325 and Worldwide gross of $113,461,527, versus $0 and $0 in our dataset. Under "Summary -> Metrics" we see that $19,153,698 (37.8% of total gross) were made during the opening weekend. Therefore at least some amount should have been included in the figures at the time of the web scraping. My classmate Jeffrey have done some digging and found that "some of the movies did not have traditional releases in Theaters, rather, they were released on streaming platforms like Netflix for instance." ([link to comment](https://www.udemy.com/course/100-days-of-code/learn/lecture/22756689#questions/20911764)  - require that you sign up and pay for the course). It is possible that revenue from streming platforms were not a part of the-numbers numbers in 2018.

If we look at "Foodfight!" at [the-numbers.com/movie/Foodfight](https://www.the-numbers.com/movie/Foodfight#tab=summary) (on January 5th, 2024) we see that "Domestic Box Office" = "n/a", and "Worldwide Box Office" = $120,323. There might have been changes since the dataset was scraped in 2018, or "n/a" values might have been interpreted (coded) as $0, which would be misleading because it is in fact a missing value. Missing values are also the case for ["The Ridiculous 6 (2015)"](https://www.the-numbers.com/movie/Ridiculous-6-The#tab=summary), which was released by Netflix.

These findings indicate that we can not distinguish between movies that made $0 and movies with unknown revenue (missing values). Therefore I would argue that the dataset is not a god basis for saying more about zero grossing films. In what follows, we should remember that earnings from streaming services are not necessarily included in the figures.

🤓 Note to self: The purpose of this notebook is to learn coding. You do not have to conduct a full investigation for each question 🔬🩺🔭

**Challenge**: How many films grossed $0 worldwide? What are the highest budget films that had no revenue internationally?

- 357. See the section above for further comments.

In [232]:
data.loc[data["USD_Worldwide_Gross"] == 0]

,Rank,Release_Date,Movie_Title,USD_Production_Budget,USD_Worldwide_Gross,USD_Domestic_Gross,USD_Profit
1,5140,1916-05-09,Intolerance,385907,0,0,-385907
6,4630,1927-12-08,Wings,2000000,0,0,-2000000
8,4240,1930-01-01,Hell's Angels,4000000,0,0,-4000000
17,4814,1936-10-20,"Charge of the Light Brigade, The",1200000,0,0,-1200000
27,4789,1941-10-28,How Green Was My Valley,1250000,0,0,-1250000
...,...,...,...,...,...,...,...
5386,2950,2018-10-08,Meg,15000000,0,0,-15000000
5387,126,2018-12-18,Aquaman,160000000,0,0,-160000000
5388,96,2020-12-31,Singularity,175000000,0,0,-175000000
5389,1119,2020-12-31,Hannibal the Conqueror,50000000,0,0,-50000000


### Filtering on Multiple Conditions

**Challenge**: Use the [`.query()` function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html) to accomplish the same thing. Create a subset for international releases that had some worldwide gross revenue, but made zero revenue in the United States. 

Hint: This time you'll have to use the `and` keyword.

### Unreleased Films

**Challenge**:
* Identify which films were not released yet as of the time of data collection (May 1st, 2018).
* How many films are included in the dataset that have not yet had a chance to be screened in the box office? 
* Create another DataFrame called data_clean that does not include these films. 

In [ ]:
# Date of Data Collection
scrape_date = pd.Timestamp('2018-5-1')

### Films that Lost Money

**Challenge**: 
What is the percentage of films where the production costs exceeded the worldwide gross revenue? 

# Seaborn for Data Viz: Bubble Charts

### Plotting Movie Releases over Time

**Challenge**: Try to create the following Bubble Chart:

<img src=https://i.imgur.com/8fUn9T6.png>



# Converting Years to Decades Trick

**Challenge**: Create a column in `data_clean` that has the decade of the release. 

<img src=https://i.imgur.com/0VEfagw.png width=650> 

Here's how: 
1. Create a [`DatetimeIndex` object](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DatetimeIndex.html) from the Release_Date column. 
2. Grab all the years from the `DatetimeIndex` object using the `.year` property.
<img src=https://i.imgur.com/5m06Ach.png width=650>
3. Use floor division `//` to convert the year data to the decades of the films.
4. Add the decades as a `Decade` column to the `data_clean` DataFrame.

### Separate the "old" (before 1969) and "New" (1970s onwards) Films

**Challenge**: Create two new DataFrames: `old_films` and `new_films`
* `old_films` should include all the films before 1969 (up to and including 1969)
* `new_films` should include all the films from 1970 onwards
* How many films were released prior to 1970?
* What was the most expensive film made prior to 1970?

# Seaborn Regression Plots

**Challenge**: Use Seaborn's `.regplot()` to show the scatter plot and linear regression line against the `new_films`. 
<br>
<br>
Style the chart

* Put the chart on a `'darkgrid'`.
* Set limits on the axes so that they don't show negative values.
* Label the axes on the plot "Revenue in \$ billions" and "Budget in \$ millions".
* Provide HEX colour codes for the plot and the regression line. Make the dots dark blue (#2f4b7c) and the line orange (#ff7c43).

Interpret the chart

* Do our data points for the new films align better or worse with the linear regression than for our older films?
* Roughly how much would a film with a budget of $150 million make according to the regression line?

# Run Your Own Regression with scikit-learn

$$ REV \hat ENUE = \theta _0 + \theta _1 BUDGET$$

**Challenge**: Run a linear regression for the `old_films`. Calculate the intercept, slope and r-squared. How much of the variance in movie revenue does the linear model explain in this case?

# Use Your Model to Make a Prediction

We just estimated the slope and intercept! Remember that our Linear Model has the following form:

$$ REV \hat ENUE = \theta _0 + \theta _1 BUDGET$$

**Challenge**:  How much global revenue does our model estimate for a film with a budget of $350 million? 